In [ ]:
#install.packages("sjmisc")
#install.packages("fdth", quiet=TRUE)

library(readxl)
library(sjmisc)
library(fdth)
library(dplyr)

In [ ]:
options(warn=-1)

In [ ]:
dataset <- read_excel("./lapaz_troncales_limpio.xlsx")
dim(dataset)

In [ ]:
str(dataset)

In [ ]:
ftable(dataset$turno )

In [ ]:
ftable(dataset$AñoCursado)

In [ ]:
ftable(dataset$NivelEnseñanza)

## Eliminamos la columna de Nivel, ya que NivelEnseñanza nos brinda más información que la anterior y están relacionadas.
## También eliminamos la columna de año ya que todos nuestros datos son del 2022, sería redundante dejarla.

In [ ]:
dataset_mas_limpio <- dplyr::select(dataset, -c(año, Nivel))

In [ ]:
head(dataset_mas_limpio)

In [ ]:
names(dataset_mas_limpio)

In [ ]:
dim(dataset_mas_limpio)

# Gráficos

In [ ]:
ftable(dataset_mas_limpio$Modalidad)

### Graficos de sectores, primaria y secundaria, para analizar la distribución de estudiantes

### Primaria

In [ ]:
dataset_mas_limpio_matricula <- dataset_mas_limpio %>% dplyr::distinct(Documento, .keep_all = TRUE)
sectores<-filter(dataset_mas_limpio_matricula, (AñoCursado == "1" |  AñoCursado == "2" |  AñoCursado == "3" |
                     AñoCursado == "4" |  AñoCursado == "5" |  AñoCursado == "6") & Modalidad == "Primaria")

In [ ]:
library(ggplot2)
pie_commute <- round(100 * table(sectores$AñoCursado) / length(sectores$AñoCursado))
pie_commute
porc <- c(pie_commute)
frec<- c(table(sectores$AñoCursado))
D1<-data.frame(frec,porc)
D1
Años<-rownames(D1)

In [ ]:
ggplot(D1, aes(x="", y=frec, fill=Años)) +
  geom_bar(stat="identity", width=1,color="white") +
  coord_polar("y", start=0)+
  theme_void()+
  geom_text(aes(label = paste0(frec,'\n', porc, ' %'),x = 1), color = "white", size=4,
            position = position_stack(vjust = 0.5))+
  scale_fill_brewer(palette="Set2")+
  labs(title="Distribucion Porcentual de la Matricula de Primaria de 2022 en
            el Departamento La Paz por Año de Cursado")

### Secundaria

In [ ]:
sectores_secu<-filter(dataset_mas_limpio_matricula, (AñoCursado == "1" |  AñoCursado == "2" |  AñoCursado == "3" |
                     AñoCursado == "4" |  AñoCursado == "5" |  AñoCursado == "6") & Modalidad == "Secundaria")

In [ ]:
pie <- round(100 * table(sectores_secu$AñoCursado) / length(sectores_secu$AñoCursado))
pie
porc <- c(pie)
frec<- c(table(sectores_secu$AñoCursado))
D1<-data.frame(frec,porc)
D1
Años<-rownames(D1)

In [ ]:
ggplot(D1, aes(x="", y=frec, fill=Años)) +
  geom_bar(stat="identity", width=1,color="white") +
  coord_polar("y", start=0)+
  theme_void()+
  geom_text(aes(label = paste0(frec,'\n', porc, ' %'),x = 1), color = "white", size=4,
            position = position_stack(vjust = 0.5))+
  scale_fill_brewer(palette="Set2")+
  labs(title="Distribucion Porcentual de la Matricula de secundaria de 2022 en
            departamento La Paz por Año de Cursado")

### Distribución por turno 

### Primaria

In [ ]:
sector_por_turno_p <- filter(dataset_mas_limpio_matricula, Modalidad == 'Primaria')

In [ ]:
sector_por_turno_p$turno <- ifelse(sector_por_turno_p$turno %in% c("INTERMEDIO", "ROTATIVO", "COMPLETO"), "OTROS", sector_por_turno_p$turno)
ftable(sector_por_turno_p$turno)

In [ ]:
pie <- round(100 * table(sector_por_turno_p$turno) / length(sector_por_turno_p$turno))
porc <- c(pie)
frec <- c(table(sector_por_turno_p$turno))
D1 <- data.frame(frec, porc)
rownames(D1) <- names(table(sector_por_turno_p$turno))
D1
turnos<-rownames(D1)

In [ ]:
pie <- round(100 * table(sector_por_turno_p$turno) / length(sector_por_turno_p$turno))
pie
porc <- c(pie)
frec<- c(table(sector_por_turno_p$turno))
D1<-data.frame(frec,porc)
D1
turnos<-rownames(D1)

In [ ]:
ggplot(D1, aes(x="", y=frec, fill=turnos)) +
  geom_bar(stat="identity", width=1,color="white") +
  coord_polar("y", start=0)+
  theme_void()+
  geom_text(aes(label = paste0(frec,'\n', porc, ' %'),x = 1), color = "white", size=4,
            position = position_stack(vjust = 0.50))+
  scale_fill_brewer(palette="Set2")+
  labs(title="Distribucion Porcentual de la Matricula de Primaria de 2022 en
            el Departamento La Paz por Turnos")

### Secundaria

In [ ]:
sector_por_turno_s <- filter(dataset_mas_limpio_matricula, Modalidad == 'Secundaria')
ftable(sector_por_turno_s$turno)

In [ ]:
sector_por_turno_s$turno <- ifelse(sector_por_turno_s$turno %in% c("DIURNO", "NOCHE", "ROTATIVO","VESPERTINO"), "OTROS", sector_por_turno_s$turno)
ftable(sector_por_turno_p$turno)
pie <- round(100 * table(sector_por_turno_s$turno) / length(sector_por_turno_s$turno))
pie
porc <- c(pie)
frec<- c(table(sector_por_turno_s$turno))
D1<-data.frame(frec,porc)
D1
Turnos<-rownames(D1)

In [ ]:
ggplot(D1, aes(x="", y=frec, fill=Turnos)) +
  geom_bar(stat="identity", width=1,color="white") +
  coord_polar("y", start=0)+
  theme_void()+
  geom_text(aes(label = paste0(frec,'\n', porc, ' %'),x = 1), color = "white", size=4,
            position = position_stack(vjust = 0.5))+
  scale_fill_brewer(palette="Set2")+
  labs(title="Distribucion Porcentual de la Matricula de Secundaria de 2022 en
            el Departamento La Paz por Turnos")

# Distribucion por sector publico y privado

In [ ]:
#Gráfico de barras de frecuencia absoluta
nombres <- c("Publica", "Privada")
escu_primaria <- filter(dataset_mas_limpio_matricula,Modalidad == "Primaria")
tabla <- table(escu_primaria$EsPrivada)
tabla
#barplot(tabla,main="Distribucion segmentado por publicas y privadas",col=rainbow(6),ylim=c(0,30000),las=2)
# Crear el gráfico de barras con nombres de columnas
barplot(tabla, 
        main = "Distribución de Notas Segmentada por Públicas y Privadas",
        col = rainbow(7), 
        ylim = c(0, 5000), 
        las = 1,
        names.arg = nombres)

In [ ]:
sector_pp <- filter(dataset_mas_limpio_matricula, Modalidad == 'Primaria')
pie <- round(100 * table(sector_pp$EsPrivada) / length(sector_pp$EsPrivada))
pie
porc <- c(pie)
frec<- c(table(sector_pp$EsPrivada))
D1<-data.frame(frec,porc)
D1

Sector<-rownames(D1)

In [ ]:
ggplot(D1, aes(x="", y=frec, fill=Sector)) +
  geom_bar(stat="identity", width=1,color="white") +
  coord_polar("y", start=0)+
  theme_void()+
  geom_text(aes(label = paste0(frec,'\n', porc, ' %'),x = 1), color = "white", size=4,
            position = position_stack(vjust = 0.5))+
    scale_fill_manual(values=c("#F8766D", "#7CAE00"), labels=c("Pública","Privada"))+
  #scale_fill_brewer(palette="Set2")+
  labs(title="Distribución Porcentual de Estudiantes de Primaria Segmentado por Sector")



In [ ]:
escu_secu<- filter(dataset_mas_limpio_matricula, Modalidad == 'Secundaria')
pie <- round(100 * table(escu_secu$EsPrivada) / length(escu_secu$EsPrivada))
pie
porc <- c(pie)
frec<- c(table(escu_secu$EsPrivada))
D1_s<-data.frame(frec,porc)
D1_s

Sector<-rownames(D1_s)

In [ ]:
ggplot(D1_s, aes(x="", y=frec, fill=Sector)) +
  geom_bar(stat="identity", width=1,color="white") +
  coord_polar("y", start=0)+
  theme_void()+
  geom_text(aes(label = paste0(frec,'\n', porc, ' %'),x = 1), color = "white", size=4,
            position = position_stack(vjust = 0.5))+
    scale_fill_manual(values=c("#F8766D", "#7CAE00"), labels=c("Pública","Privada"))+
  #scale_fill_brewer(palette="Set2")+
  labs(title="Distribución Porcentual de Estudiantes de Secundaria Segmentado por Sector")

# Segmentado por nivel, año y materia de la variable promedio

### Materia matemática, primer año - Primaria

In [ ]:
primaria_Mat_1<-filter(dataset_mas_limpio, AñoCursado == "1" & asignatura=="MATEMÁTICA" & Modalidad == "Primaria")

boxplot(primaria_Mat_1[!is.na(primaria_Mat_1$Promedio),]$Promedio,
        whisklty = 2,       # Tipo línea bigotes
        whisklwd = 2,       # Ancho línea bigotes
        whiskcol = "red",   # Color bigotes
        staplelty = 3,      # Tipo línea barras
        staplelwd = 2,      # Ancho línea barras
        staplecol = "blue", # Color barras
        main ="Cajas y bigotes de Promedio para Matemáticas - Primaria", # Tìtulo
        horizontal = TRUE
       )

### Materia Lengua, primer año - Primaria

In [ ]:
primaria_len_1<-filter(dataset_mas_limpio, AñoCursado == "1" & asignatura=="LENGUA" & Modalidad == "Primaria")

boxplot(primaria_len_1[!is.na(primaria_len_1$Promedio),]$Promedio,
        whisklty = 2,       # Tipo línea bigotes
        whisklwd = 2,       # Ancho línea bigotes
        whiskcol = "red",   # Color bigotes
        staplelty = 3,      # Tipo línea barras
        staplelwd = 2,      # Ancho línea barras
        staplecol = "blue", # Color barras
        main ="Cajas y Bigotes de Promedio para Lengua - Primaria", # Tìtulo
        horizontal = TRUE
       )

### Materia matemática, primer año - Secundaria



In [ ]:
secundaria_Mat_1<-filter(dataset_mas_limpio, AñoCursado == 1 & asignatura=="MATEMATICA" & Modalidad == "Secundaria")

boxplot(secundaria_Mat_1[!is.na(secundaria_Mat_1$Promedio),]$Promedio,
        whisklty = 2,       # Tipo línea bigotes
        whisklwd = 2,       # Ancho línea bigotes
        whiskcol = "red",   # Color bigotes
        staplelty = 3,      # Tipo línea barras
        staplelwd = 2,      # Ancho línea barras
        staplecol = "blue", # Color barras
        main ="Cajas y Bigotes de Promedio para Matemática - Secundaria", # Tìtulo
        horizontal = TRUE
       )

### Materia Lengua y Literatura, primer año - Secundaria

In [ ]:
secundaria_len_1<-filter(dataset_mas_limpio, AñoCursado == "1" & asignatura=="LENGUA Y LITERATURA" & Modalidad == "Secundaria")

boxplot(secundaria_len_1[!is.na(secundaria_len_1$Promedio),]$Promedio,
        whisklty = 2,       # Tipo línea bigotes
        whisklwd = 2,       # Ancho línea bigotes
        whiskcol = "red",   # Color bigotes
        staplelty = 3,      # Tipo línea barras
        staplelwd = 2,      # Ancho línea barras
        staplecol = "blue", # Color barras
        main ="Cajas y Bigotes de Promedio para Lengua y Literatura - Secundaria", # Tìtulo
        horizontal = TRUE
       )

## Histogramas segmentados por nivel, año y materia de la variable promedio para matemática y lengua

## Primaria
### Materia: matemática

In [ ]:
for (i in 1:6)
{
    print(paste0("Año cursado: ", i))
    primaria_mat <- filter(dataset_mas_limpio, AñoCursado == i & asignatura == "MATEMÁTICA" & Modalidad == 'Primaria')
    histo_p_1 <- fdt(primaria_mat$Promedio,breaks="Sturges",na.rm=TRUE)
    histo_p <- ggplot(primaria_mat[!is.na(primaria_mat$Promedio),],aes(x= Promedio))+
        geom_histogram(fill="pink",colour="brown",name="matemática", bins=10)+
            scale_y_continuous(breaks = seq(0, 500,50), name = "Frecuencia Absoluta") +
      scale_x_continuous(breaks = seq(0,10,1), name = "Notas" ) + geom_vline(aes(xintercept = median(Promedio),
                      color = "mediana"),
                  linetype = "dashed",
                  size = 1) +
       geom_vline(aes(xintercept = mean(Promedio),
                      color = "media"),
                  linetype = "dashed",
                  size = 1) +
       geom_vline(
         aes(xintercept = quantile(Promedio, 0.25),
             color = "cuantil_25"),
         linetype = "dashed",
         size = 1
       ) +
       geom_vline(
         aes(xintercept = quantile(Promedio, 0.75),
             color = "cuantil_75"),
         linetype = "dashed",
         size = 1
       )  +
       scale_color_manual(
         name = "estadisticos",
         values = c(
           mediana = "blue",
           media = "red",
           cuantil_25 = "black",
           cuantil_75 = "orange"
         )
       ) +
       xlab("Longitud") + ylab("n") +
    ggtitle(paste0("Primaria - Histograma de promedio de Matemática - ", i, " año.")) +
   theme(plot.title = element_text(hjust = 0.5))
    print(histo_p)
}


### Materia: Lengua

In [ ]:
for (i in 1:6)
{
    print(paste0("Año cursado: ", i))
    primaria_len <- filter(dataset_mas_limpio, AñoCursado == i & asignatura == "LENGUA" & Modalidad == 'Primaria')
    histo_p_1 <- fdt(primaria_len$Promedio,breaks="Sturges",na.rm=TRUE)
    histo_p <- ggplot(primaria_len[!is.na(primaria_len$Promedio),],aes(x= Promedio))+
        geom_histogram(fill="pink",colour="brown",name="lengua", bins=10)+
            scale_y_continuous(breaks = seq(0, 500,50), name = "Frecuencia Absoluta") +
      scale_x_continuous(breaks = seq(0,10,1), name = "Notas" ) + geom_vline(aes(xintercept = median(Promedio),
                      color = "mediana"),
                  linetype = "dashed",
                  size = 1) +
       geom_vline(aes(xintercept = mean(Promedio),
                      color = "media"),
                  linetype = "dashed",
                  size = 1) +
       geom_vline(
         aes(xintercept = quantile(Promedio, 0.25),
             color = "cuantil_25"),
         linetype = "dashed",
         size = 1
       ) +
       geom_vline(
         aes(xintercept = quantile(Promedio, 0.75),
             color = "cuantil_75"),
         linetype = "dashed",
         size = 1
       )  +
       scale_color_manual(
         name = "estadisticos",
         values = c(
           mediana = "blue",
           media = "red",
           cuantil_25 = "black",
           cuantil_75 = "orange"
         )
       ) +
       xlab("Longitud") + ylab("n") +
    ggtitle(paste0("Primaria - Histograma de promedio de Lengua - ", i, " año.")) +
   theme(plot.title = element_text(hjust = 0.5))
    print(histo_p)
}


## Secundaria
### Materia: matemática

In [ ]:
for (i in 1:7)
{
    print(paste0("Año cursado: ", i))
    secundaria_mat<- filter(dataset_mas_limpio, AñoCursado == i & asignatura == "MATEMATICA" & Modalidad == 'Secundaria')
    histo_s_1 <- fdt(secundaria_mat$Promedio,breaks="Sturges",na.rm=TRUE)
    histo_s <- ggplot(secundaria_mat[!is.na(secundaria_mat$Promedio),],aes(x= Promedio))+
        geom_histogram(fill="pink",colour="brown",name="matematica", bins=10)+
            scale_y_continuous(breaks = seq(0, 500,50), name = "Frecuencia Absoluta") +
      scale_x_continuous(breaks = seq(0,10,1), name = "Notas" ) + geom_vline(aes(xintercept = median(Promedio),
                      color = "mediana"),
                  linetype = "dashed",
                  size = 1) +
       geom_vline(aes(xintercept = mean(Promedio),
                      color = "media"),
                  linetype = "dashed",
                  size = 1) +
       geom_vline(
         aes(xintercept = quantile(Promedio, 0.25),
             color = "cuantil_25"),
         linetype = "dashed",
         size = 1
       ) +
       geom_vline(
         aes(xintercept = quantile(Promedio, 0.75),
             color = "cuantil_75"),
         linetype = "dashed",
         size = 1
       )  +
       scale_color_manual(
         name = "estadisticos",
         values = c(
           mediana = "blue",
           media = "red",
           cuantil_25 = "black",
           cuantil_75 = "orange"
         )
       ) +
       xlab("Longitud") + ylab("n") +
        ggtitle(paste0("Secundaria - Histograma de promedio de Matemática - ", i, " año.")) +
       theme(plot.title = element_text(hjust = 0.5))
    print(histo_s)
}

### Materia: Lengua y literatura

In [ ]:
for (i in 1:7)
{
    print(paste0("Año cursado: ", i))
    secundaria_len<- filter(dataset_mas_limpio, AñoCursado == i & asignatura == "LENGUA Y LITERATURA" & Modalidad == 'Secundaria')
    histo_s_1 <- fdt(secundaria_len$Promedio,breaks="Sturges",na.rm=TRUE)
    histo_s <- ggplot(secundaria_len[!is.na(secundaria_len$Promedio),],aes(x= Promedio))+
        geom_histogram(fill="pink",colour="brown",name="Lengua y literatura", bins=10)+
            scale_y_continuous(breaks = seq(0, 500,50), name = "Frecuencia Absoluta") +
      scale_x_continuous(breaks = seq(0,10,1), name = "Notas" ) + geom_vline(aes(xintercept = median(Promedio),
                      color = "mediana"),
                  linetype = "dashed",
                  size = 1) +
       geom_vline(aes(xintercept = mean(Promedio),
                      color = "media"),
                  linetype = "dashed",
                  size = 1) +
       geom_vline(
         aes(xintercept = quantile(Promedio, 0.25),
             color = "cuantil_25"),
         linetype = "dashed",
         size = 1
       ) +
       geom_vline(
         aes(xintercept = quantile(Promedio, 0.75),
             color = "cuantil_75"),
         linetype = "dashed",
         size = 1
       )  +
       scale_color_manual(
         name = "estadisticos",
         values = c(
           mediana = "blue",
           media = "red",
           cuantil_25 = "black",
           cuantil_75 = "orange"
         )
       ) +
       xlab("Longitud") + ylab("n") +
        ggtitle(paste0("Secundaria - Histograma de promedio de Lengua y Literatura - ", i, " año.")) +
       theme(plot.title = element_text(hjust = 0.5))
    print(histo_s)}


In [ ]:
secundaria_len<- filter(dataset_mas_limpio, AñoCursado == 7 & asignatura == "LENGUA Y LITERATURA" & Modalidad == 'Secundaria')
ftable(secundaria_len$Promedio)

## Guía 4

### a. ¿Son las correlaciones iguales?
### b. A partir de lo que vimos en teoría, 
### ¿usted podría informar todas las correlaciones comocorrelaciones lineales fuertes?
### c. ¿Qué importancia tiene el graficar los datos?

In [ ]:
library(dplyr)
library(gridExtra)
anscombe_data = read.delim(file="https://goo.gl/mVLz5L", header = T)
# Calcular la correlación
cor_1 = cor(anscombe_data$x1, anscombe_data$y1)
cor_2 = cor(anscombe_data$x2, anscombe_data$y2)
cor_3 = cor(anscombe_data$x3, anscombe_data$y3)
cor_4 = cor(anscombe_data$x4, anscombe_data$y4)
# Definir la función
plot_anscombe <- function(x, y, value, type)
{ # 'anscombe_data' es una variable global, esto es
# una mala práctica de programación ;)
p=ggplot(anscombe_data, aes_string(x,y)) +
geom_smooth(method='lm', fill=NA) +
geom_point(aes(colour=factor(1),
fill = factor(1)),
shape=21, size = 2
) +
ylim(2, 13) +
xlim(4, 19) +
theme_minimal() +
theme(legend.position="none") +
annotate("text",
x = 12,
y =4.5,
label =
sprintf("%s: %s",
type,
round(value,2)
)
)
return(p)
}
# Graficar en una cuadrícula de 2x2
grid.arrange(plot_anscombe("x1", "y1", cor_1, "Correlación"),
plot_anscombe("x2", "y2", cor_2, "Correlación"),
plot_anscombe("x3", "y3", cor_3, "Correlación"),
plot_anscombe("x4", "y4", cor_4, "Correlación"),
ncol=2, nrow=2)


## Punto 4
### Comparación mediante histograma

### Secundaria - primer año

In [ ]:
secundaria_Mat_1<-filter(dataset_mas_limpio, AñoCursado == 1 & asignatura=="MATEMATICA" & Modalidad == "Secundaria")

### Comparación mediante Boxplot

In [ ]:
g2 <- ggplot(secundaria_Mat_1, aes(factor(EsPrivada), Promedio))
g2 + geom_boxplot(aes(fill = factor(EsPrivada)))

## Punto 5

### Comparación de promedios de matemáticas de los diferentes años en distintos turnos

In [ ]:
ftable(filter(dataset_mas_limpio, asignatura == "MATEMÁTICA" & Modalidad == "Primaria")$turno)

In [ ]:
ftable(filter(dataset_mas_limpio, asignatura == "LENGUA" & Modalidad == "Primaria")$turno)

### Primaria

In [ ]:
#g2 <- ggplot(filter(dataset_mas_limpio,asignatura=="MATEMÁTICA" & Modalidad=='Primaria'),aes(factor(turno),Promedio))
#g2 + geom_boxplot(aes(fill = factor(turno))
para_nombres<- filter(dataset_mas_limpio,asignatura=="MATEMÁTICA" & Modalidad=='Primaria')
Turnos <- factor(para_nombres$turno)

g2 <- ggplot(filter(dataset_mas_limpio,asignatura=="MATEMÁTICA" & Modalidad=='Primaria'),aes(Turnos,Promedio))
g2 + 
  geom_boxplot(aes(fill = Turnos)) +
  labs(title = "Distribución de Notas de Matemática por Turnos en Educación Primaria")


### Secundaria

In [ ]:
ftable(filter(dataset_mas_limpio, asignatura == "MATEMATICA" & Modalidad == "Secundaria")$turno)

In [ ]:
ftable(filter(dataset_mas_limpio, asignatura == "LENGUA Y LITERATURA" & Modalidad == "Secundaria")$turno)

In [ ]:
#g2 <- ggplot(filter(dataset_mas_limpio,asignatura=="MATEMATICA" & Modalidad=='Secundaria'),aes(factor(turno),Promedio))
#g2 + geom_boxplot(aes(fill = factor(turno)))
v<- filter(dataset_mas_limpio,asignatura=="MATEMATICA" & Modalidad=='Secundaria')
Turnos <- factor(v$turno)

g2 <- ggplot(filter(dataset_mas_limpio,asignatura=="MATEMATICA" & Modalidad=='Secundaria'),aes(Turnos,Promedio))
g2 + 
  geom_boxplot(aes(fill = Turnos)) +
  labs(title = "Distribución de Notas de Matemática por Turnos en Educación Secundaria")

### Comparación de los promedios de lengua en distintos turnos

In [ ]:
v<- filter(dataset_mas_limpio,asignatura=="LENGUA" & Modalidad=='Primaria')
Turnos <- factor(v$turno)

g2 <- ggplot(filter(dataset_mas_limpio,asignatura=="LENGUA" & Modalidad=='Primaria'),aes(Turnos,Promedio))
g2 + 
  geom_boxplot(aes(fill = Turnos)) +
  labs(title = "Distribución de Notas de Lengua por Turnos en Educación Primaria")

In [ ]:
v<- filter(dataset_mas_limpio,asignatura=="LENGUA Y LITERATURA" & Modalidad=='Secundaria')
Turnos <- factor(v$turno)

g2 <- ggplot(filter(dataset_mas_limpio,asignatura=="LENGUA Y LITERATURA" & Modalidad=='Secundaria'),aes(Turnos,Promedio))
g2 + 
  geom_boxplot(aes(fill = Turnos)) +
  labs(title = "Distribución de Notas de Lengua y Literatura por Turnos en Secundaria")

## Punto 6

In [ ]:
#diagrama de dispersión
#deben tener misma dimension
rm(df)
dim(dplyr::select(secundaria_Mat_1,"Documento","Promedio","EsPrivada"))
dim(dplyr::select(filter(dataset_mas_limpio,asignatura=="BIOLOGIA" & AñoCursado == 1),"Documento","Promedio","EsPrivada"))
df<- dplyr::select(secundaria_Mat_1,"Documento","Promedio","EsPrivada") %>% inner_join(dplyr::select(filter(dataset_mas_limpio,asignatura=="BIOLOGIA" & AñoCursado== 1),"Documento","Promedio","EsPrivada"),by="Documento")
df<-rename(df,Prom_Mat=Promedio.x, Prom_Bio=Promedio.y,EsPrivada=EsPrivada.x)
head(df)
plot(df$Prom_Mat, df$Prom_Bio)

In [ ]:
ggplot(data=df, aes(x=Prom_Mat,y=Prom_Bio))+geom_point(aes(colour=EsPrivada)
)+xlab("Longitud")+ylab("ancho")

### Diagrama de dispersión para todos los años de secundaria de matematica vs biologia

In [ ]:
#diagrama de dispersión
#deben tener misma dimension
rm(df2)

secundaria_mat_todos_no_7 <- dplyr::filter(dataset_mas_limpio, asignatura == "MATEMATICA" & Modalidad == "Secundaria" & AñoCursado %in% c(1, 2, 3, 4, 5, 6))
dim(dplyr::select(secundaria_mat_todos_no_7,"Documento","Promedio"))
dim(dplyr::select(filter(dataset_mas_limpio,asignatura=="BIOLOGIA"),"Documento","Promedio"))
df2<- dplyr::select(secundaria_mat_todos_no_7,"Documento","Promedio","AñoCursado") %>% inner_join(dplyr::select(filter(dataset_mas_limpio,asignatura=="BIOLOGIA"),"Documento","Promedio","AñoCursado"),by="Documento")
df2<-rename(df2,Prom_Mat=Promedio.x, Prom_Bio=Promedio.y, AñoCursado=AñoCursado.x)
df2$AñoCursado <- as.factor(df2$AñoCursado)
head(df2)
ggplot(data=df2, aes(x=Prom_Mat,y=Prom_Bio))+ geom_point(aes(colour=AñoCursado)
)+xlab("Promedio Biología")+ylab("Promedio Matemáticas")+scale_color_manual(values = c("red", "blue", "green", "purple", "pink", "yellow"))

### Como observamos claramente, el año más prometedor para una posible correlación es primero. Por lo que a partir de este momento centraremos el estudio en este año.

In [ ]:
df2_primero <- dplyr::filter(df2, AñoCursado == "1")
head(df2_primero)

### Graficamos nuevamamente solamente primer año

In [ ]:
ggplot(data=df2_primero, aes(x=Prom_Mat,y=Prom_Bio))+geom_point()+ 
       geom_smooth(method = "lm")+xlab("Promedio Biología")+ylab("Promedio Matemáticas")

### Matriz de Varianza

In [ ]:
dataM <- na.omit(data.frame(df2_primero$Prom_Mat, df2_primero$Prom_Bio)) 
dataM <-rename(dataM,Prom_Mat=df2_primero.Prom_Mat, Prom_Bio=df2_primero.Prom_Bio)
var(dataM)

### Primero debemos verificar normalidad de las variables a utilizar, Prom_Mat y Prom_Bio
### Usamos Lilliefors (Kolmogorov-Smirnov) para realizarlo ya que nuestra muestra es mayor a 50

In [ ]:
#conda install -c conda-forge r-nortest
library(nortest)
lillie.test(x = dataM$Prom_Mat)
lillie.test(x = dataM$Prom_Bio)

### El p-value para ambas variables es menor a 0.05, lo que significa que rechazamos H0 y podemos decir que las variables no presentan una distribución normal, por lo que usaremos el método de Spearman para comprobar linealidad entre las mismas

In [ ]:
cor_spearman <- cor(dataM$Prom_Mat, dataM$Prom_Bio, method = "spearman")
cor_spearman

### Podemos ver gráficamente el resultado también

In [ ]:
library("PerformanceAnalytics")
chart.Correlation(dataM, histogram = TRUE, method = "spearman")
mtext("Gráfico de Relación entre Promedios de Matemáticas y Biología", side=3, line=3)

### Podemos decir que las variables presentan una correlación buena, esto significa que si a un alumno le va bien en matemáticas, le irá bien en biología y lo mismo inversamente.

In [ ]:
per <- cor(dataM$Prom_Mat, dataM$Prom_Bio, method = "pearson")
ken <- cor(dataM$Prom_Mat, dataM$Prom_Bio, method = "kendall")
spe <- cor(dataM$Prom_Mat, dataM$Prom_Bio, method = "spearman")

data <- data.frame(name=c("pearson","kendall","spearman") , value=c(per,ken,spe))

data

In [ ]:
library(corrplot)
corrplot.mixed(cor(dataM, method = "spearman"),addshade="all")

### Añadimos lengua como tercera variable de estudio

In [ ]:
rm(df3)
df3 <- dplyr::select(secundaria_len_1,"Documento","Promedio", "AñoCursado")

df2<- df2 %>% inner_join(df3,by="Documento")

In [ ]:
head(df2)

In [ ]:

dataN <-rename(df2,Prom_Len=Promedio)
dataN <- na.omit(dataN) 
dataN <-dplyr::select(dataN,Prom_Mat,Prom_Bio,Prom_Len)
dim(dataN)
head(dataN)

In [ ]:
library(ppcor)
pcor(dataM, method = c("spearman"))

In [ ]:
pcor(dataN, method = c("spearman"))

In [ ]:
ggm::pcor(c("Prom_Mat","Prom_Bio","Prom_Len"), cov(dataN))
pcor1<-ggm::pcor(c("Prom_Mat","Prom_Bio","Prom_Len"), cov(dataN))

In [ ]:
library("pwr")
pwr.r.test(n = length(row.names(dataN)), r = pcor1, sig.level = .05, power = NULL)
